In [9]:
from torchvision import utils
from unet_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [10]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=2,
                          num_workers=10,
                          shuffle=True)

In [11]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        torch.nn.init.zeros_(m.bias.data)
        
epochs     = 10
criterion = nn.CrossEntropyLoss()
fcn_model = UNet(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Access to GPU, you did it")
    fcn_model = fcn_model.cuda()

Access to GPU, you did it


In [17]:
for name, param in fcn_model.named_parameters():
    print(name)

classifier.weight
classifier.bias


In [12]:

    
def train():
    losses = []
    p_accs = []
    iou_accs = []
    for epoch in range(epochs+1):
        losses_epoch = []
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()
                labels = Y.cuda()
            else:
                inputs, labels = X.cpu(), Y.cpu()

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            losses_epoch.append(loss.item())
            loss.backward()
            optimizer.step()
            
            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        losses.append(np.mean(np.array(losses_epoch)))
        p_acc,iou_acc = val(epoch)
        p_accs.append(p_acc)
        iou_accs.append(iou_acc)
        fcn_model.train()
        if epoch%10 == 0:
            torch.save(fcn_model, 'best_model')
            np.save("losses",np.array(losses))
            np.save("p_acc",np.array(p_accs))
            np.save("iou_acc",np.array(iou_accs))

def val(epoch):
    # Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    p_acc = 0
    iou_acc = 0
    count = 0
    for iter, (X, tar, Y) in enumerate(val_loader):
        if use_gpu:
            X = X.cuda()
            tar = tar.cuda()
        else:
            X,tar = X.cpu(), tar.cpu()
        p, iou = fcn_model.eval(X, tar)
        p_acc += p
        iou_acc += np.mean(np.array(iou))
        count += 1
    print("Epoch {}: Pixel Acc: {}, IOU Acc: {}".format(epoch, p_acc/count, iou_acc/count))
    return p_acc/count, iou_acc/count
        
    
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    p_acc = 0
    iou_acc = 0
    count = 0
    for iter, (X, tar, Y) in enumerate(test_loader):
        if use_gpu:
            X = X.cuda()
            tar = tar.cuda()
        else:
            X,tar = X.cpu(), tar.cpu()
        p, iou = fcn_model.eval(X, tar)
        p_acc += p
        iou_acc += np.mean(np.array(iou))
        count += 1
    print("Pixel Acc: {}, IOU Acc: {}".format(p_acc/count, iou_acc/count))
    return p_acc/count, iou_acc/count
    
if __name__ == "__main__":
#    val(0)# show the accuracy before training
#     test()
    train()
    print("Testing ")
    test()

Access to GPU, you did it


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same